In [1]:
from pennylane import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
sys.path.insert(0, '/Users/tak/Github/QEmbedding/')
import Hybrid_nn
import torch
from torch import nn
import data
import pennylane as qml
import embedding

In [2]:
from qiskit import IBMQ

# KQC Token
#TOKEN = "b5b31be2bb5237d52e8bdef143913cd62425eadcbda5c56c6f8ed8d067573ac0e07aa41d23d0bfcc880882a4142e2f812dad876fb3208d5c5f8f321be6626cce"
# Yonsei Token
TOKEN = "41bc5ff592064fefedbb4e5f949e42a9ded7835d2b14e69147aaab22b911ea16e1d95f6929e9b1fadf6c8e412253d50f820c25cd6c45be588d31910f2c4fd96b"


IBMQ.save_account(TOKEN, overwrite=True)
IBMQ.load_account() # Load account from disk


#KQC Provider
#provider = IBMQ.get_provider(hub='ibm-q-kqc', group='internal', project='reservations')
#provider = IBMQ.get_provider(hub='ibm-q-kqc', group='kqc-research', project='kernel-experimen')

#Yonsei Provider
provider = IBMQ.get_provider(hub='ibm-q-yonsei', group='internal', project='reservations')




backend = "ibm_lagos"

dev = qml.device(
    'qiskit.ibmq',
    wires=4,
    shots=1024,
    backend=backend,
    provider=provider
)


PATH_Model2 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Real/Model2 pre-train/Model2_48.pt'
PATH_Model3 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Real/Model3 pre-train/Model3_48.pt'


In [3]:
class x_transform2(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack2 = nn.Sequential(
            nn.Linear(4, 12),
            nn.ReLU(),
            nn.Linear(12, 12),
            nn.ReLU(),
            nn.Linear(12, 7)
        )
        
    def forward(self, x):
        x = self.linear_relu_stack2(x)
        return x.detach().numpy()


class x_transform3(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Layer2: 14 * 14 -> 7 * 7
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Fully connected Layers 7 * 7 -> 7
        self.fc = torch.nn.Linear(7 * 7, 7, bias=True)
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(-1, 7 * 7)
        x = self.fc(x)
        return x.detach().numpy()

model2 = x_transform2()
model3 = x_transform3()

In [4]:
def statepreparation(x, Trained):
    if Trained == False:
        embedding.Noisy_Four_QuantumEmbedding1(x)
    elif Trained == 'Model2':
        model2.load_state_dict(torch.load(PATH_Model2))
        x = model2(torch.tensor(x))
        embedding.Noisy_Four_QuantumEmbedding2(x)
    elif Trained == 'Model3':
        model3.load_state_dict(torch.load(PATH_Model3))
        x = model3(torch.tensor(x))
        embedding.Noisy_Four_QuantumEmbedding2(x[0])
    

@qml.qnode(dev)
def Noisy_QCNN_classifier(params, x, Trained):
    param1 = params[0:2]
    param2 = params[2:4]

    statepreparation(x, Trained)
    embedding.U_TTN(param1, wires=[0,1])
    embedding.U_TTN(param1, wires=[3,2])
    embedding.U_TTN(param2, wires=[1,2])
    return qml.expval(qml.PauliZ(2))

def accuracy_test(predictions, labels):
    acc = 0
    for l, p in zip(labels, predictions):
        if np.abs(l - p) < 1:
            acc = acc + 1
    return acc / len(labels)

# QCNN accuracy / Model2 IBMQ_Jakarta

QCNN accuracy is experimented on ibm_lagos.

In [7]:

feature_reduction = 'PCA4'
classes = [0,1]
X_train, X_test, Y_train, Y_test = data.data_load_and_process('mnist', feature_reduction=feature_reduction, classes=classes)
X_train, X_test, Y_train, Y_test = X_train[:500], X_test[:500], Y_train[:500], Y_test[:500]
X_train, X_test = torch.tensor(X_train).to(torch.float32), torch.tensor(X_test).to(torch.float32)
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]


weights_Model2_Jakarta = [-0.01100251, 1.22128094, -1.86724874, 0.11109528 ]
prediction_Model2_Jakarta = [Noisy_QCNN_classifier(weights_Model2_Jakarta, x, Trained='Model2') for x in X_test]
accuracy_Model2_Jakarta = accuracy_test(prediction_Model2_Jakarta, Y_test)

/var/folders/ym/ry24dhs911s7zbqfbhs5ns2m0000gn/T/ipykernel_8182/1434021917.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = model2(torch.tensor(x))


In [10]:
print(f"Accuracy with Model2_Jakarta pre-training: {accuracy_Model2_Jakarta}")

Accuracy with Model2_Jakarta pre-training: 0.964


# QCNN accuracy / Model3 IBMQ_Jakarta

QCNN accuracy is experimented on ibm_lagos

In [5]:
feature_reduction = False
classes = [0,1]
X_train, X_test, Y_train, Y_test = data.data_load_and_process('mnist', feature_reduction=feature_reduction, classes=classes)
X_train, X_test, Y_train, Y_test = X_train[:500], X_test[:500], Y_train[:500], Y_test[:500]
X_train, X_test = torch.tensor(X_train).to(torch.float32), torch.tensor(X_test).to(torch.float32)
X_train, X_test = X_train.permute(0, 3, 1, 2), X_test.permute(0, 3, 1, 2)
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]


weights_Model3_Jakarta = [2.01256055, 0.51212296, 2.11959629, -0.46627558]
prediction_Model3_Jakarta = [Noisy_QCNN_classifier(weights_Model3_Jakarta, x, Trained='Model3') for x in X_test]
accuracy_Model3_Jakarta = accuracy_test(prediction_Model3_Jakarta, Y_test)

/var/folders/ym/ry24dhs911s7zbqfbhs5ns2m0000gn/T/ipykernel_8522/1760356466.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = model3(torch.tensor(x))


In [7]:
print(f"Accuracy with Model3_Jakarta pre-training: {accuracy_Model3_Jakarta}")

Accuracy with Model3_Jakarta pre-training: 0.92
